In [145]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [146]:
DataFiles = os.listdir('CDC dataset')
display(len(DataFiles))
#display(DataFiles)

37


We first use one of the .csv files to extract indices (or IDs) with which all the files will be merged.
The process should yield a data frame whose size is equivalent to 3226 counties and 3 features, namely cnty_fips, 
county, and state.

In [147]:
file_name = DataFiles[0]
filepath = 'CDC dataset' + "/" + file_name
# display(filepath)
df = pd.read_csv(filepath)
df = df[["cnty_fips", "display_name"]]
# display(df.shape)

split_columns = df["display_name"].astype(str).str.split(pat=",", n=1, expand=True)
split_columns.head()
df["county"] = split_columns.iloc[:,0].str.strip('\"')
df["state"] = split_columns.iloc[:,1].str.strip('\"')
df["state"] = df["state"].str.slice(start = 2, stop = 4)

df.drop(columns = ["display_name"], inplace = True)

df.head()

,cnty_fips,county,state
0,2013,Aleutians East,AK
1,2016,Aleutians West,AK
2,2020,Anchorage,AK
3,2050,Bethel,AK
4,2060,Bristol Bay,AK


Now we extract the value feature from all the .csv files, while appending the data to df. We make sure to rename the value with the file name during each iteration. This process should yield a bigger dataframe with dimension equal to 3226 by 40. The 40 columns represent cnty_fips, county, state, and the values for all the 37 factors, extracted from the .csv files.

In [148]:
for idx in range(len(DataFiles)):       
    file_name = DataFiles[idx]
    filepath = 'CDC dataset' + "/" + file_name 
    df_idx = pd.read_csv(filepath)
    df_idx = df_idx.rename(columns = {"Value": file_name.strip(".csv") })
    df_idx = df_idx.drop(columns = ["display_name","theme_range"])
    # Now join this file to the existing Dataframe   
    df = df.merge(df_idx, on = "cnty_fips", how = "left")

df1 = df.set_index("cnty_fips")

display(df1.head())
df1.shape

,county,state,HEALTHCARE (Cardiac Intensive Care Units),HEALTHCARE (Cardiac Rehab. Use -Completion Rate Among Initiators),HEALTHCARE (Cardiac Rehab. Use -Eligibility Rate),HEALTHCARE (Cardiac Rehab. Use -Mean Number of Sessions Among Initiators),HEALTHCARE (Cardiac Rehab. Use -Participation Rate Among Eligible),HEALTHCARE (Cardiac Rehabilitation Units),HEALTHCARE (Cholesterol Screening),HEALTHCARE (Cholesterol-Lowering Medication - Nonadherence - All race,HEALTHCARE (CVD Physicians),HEALTHCARE (Emergency Services Centers),HEALTHCARE (Hospitals #),HEALTHCARE (Insurance Status),HEALTHCARE (Neurological Services),HEALTHCARE (Neurologists),HEALTHCARE (Neurosurgeons),HEALTHCARE (Pharmacies and Drug Stores),HEALTHCARE (Primary Care Physicians),Heart Disease Prevalence,RISK (Diagnosed Diabetes),RISK (High Cholesterol Awareness),RISK (Leisure Time Physical Inactivity),RISK (Obesity),RISK (SMOKING),SOCIAL (Air Quality PM2.5 Level),SOCIAL (Demo. Aged 65 and Older),SOCIAL (Demo. American Indian or Alaska Native),SOCIAL (Demo. Asian or Native Hawaiian or Other Pacific Island),SOCIAL (Demo. Black),SOCIAL (Demo. Hispanic or Latino),SOCIAL (Demo. TotaL Population),SOCIAL (Demo. White),SOCIAL (Income Inequality),SOCIAL (Median Household Income),SOCIAL (Park Access),SOCIAL (Poverty %),SOCIAL (Unemployment Rate),SOCIAL (Urban - Rural Status)
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013,Aleutians East,AK,0,-1.0,-1.0,-1.0,-1.0,0,72.3,-1.0,-1.0,0.0,0,25.6,0,-1.0,-1.0,0.0,-1.0,6.3,14.9,33.2,26.7,50.9,21.8,-1.0,9.4,45.8,20.6,3.2,9.0,3385,13.2,0.407,0.407,66,14.8,2.3,4
2016,Aleutians West,AK,0,-1.0,-1.0,-1.0,-1.0,0,72.9,-1.0,-1.0,0.0,0,17.6,0,-1.0,-1.0,0.0,2.9,5.3,8.2,32.8,27.2,41.5,20.4,-1.0,6.1,10.4,42.2,3.3,14.0,5698,23.8,0.351,0.351,86,7.6,3.1,4
2020,Anchorage,AK,1,35.4,13.5,23.6,24.3,1,72.5,18.7,13.9,5.1,4,12.1,2,24.3,29.2,5.1,0.9,5.1,8.6,32.1,17.0,30.1,17.3,6.4,10.5,7.3,11.8,5.2,9.1,293531,57.9,0.430,0.430,83,9.5,4.4,3
2050,Bethel,AK,0,-1.0,14.4,-1.0,-1.0,0,65.5,60.0,-1.0,0.0,1,17.9,0,-1.0,-1.0,0.0,1.0,9.1,7.2,31.9,28.1,33.9,35.8,-1.0,7.2,82.4,1.2,0.7,2.5,18134,9.7,0.450,0.450,76,23.5,11.5,4
2060,Bristol Bay,AK,0,-1.0,-1.0,-1.0,-1.0,0,75.1,-1.0,-1.0,0.0,0,16.2,0,-1.0,-1.0,0.0,-1.0,7.3,16.1,34.7,26.8,41.8,21.9,-1.0,9.9,34.1,0.4,0.5,3.6,875,50.8,0.392,0.392,30,9.8,5.3,4


(3226, 39)

In [149]:
df1.columns

Index(['county', 'state', 'HEALTHCARE (Cardiac Intensive Care Units)',
       'HEALTHCARE (Cardiac Rehab. Use -Completion Rate Among Initiators)',
       'HEALTHCARE (Cardiac Rehab. Use -Eligibility Rate)',
       'HEALTHCARE (Cardiac Rehab. Use -Mean Number of Sessions Among Initiators)',
       'HEALTHCARE (Cardiac Rehab. Use -Participation Rate Among Eligible)',
       'HEALTHCARE (Cardiac Rehabilitation Units)',
       'HEALTHCARE (Cholesterol Screening)',
       'HEALTHCARE (Cholesterol-Lowering Medication - Nonadherence - All race',
       'HEALTHCARE (CVD Physicians)',
       'HEALTHCARE (Emergency Services Centers)', 'HEALTHCARE (Hospitals #)',
       'HEALTHCARE (Insurance Status)', 'HEALTHCARE (Neurological Services)',
       'HEALTHCARE (Neurologists)', 'HEALTHCARE (Neurosurgeons)',
       'HEALTHCARE (Pharmacies and Drug Stores)',
       'HEALTHCARE (Primary Care Physicians)', 'Heart Disease Prevalence',
       'RISK (Diagnosed Diabetes)', 'RISK (High Cholesterol Awareness)

In [150]:
replace_dict = {'HEALTHCARE (Cardiac Intensive Care Units)': "cardiac_intensive_care_units",
                'HEALTHCARE (Cardiac Rehab. Use -Completion Rate Among Initiators)': "cardiac_rehab_completion" ,
                'HEALTHCARE (Cardiac Rehab. Use -Eligibility Rate)': "cardiac_rehab_eligibility",
                'HEALTHCARE (Cardiac Rehab. Use -Mean Number of Sessions Among Initiators)': "cardiac_rehab_sessions",
                'HEALTHCARE (Cardiac Rehab. Use -Participation Rate Among Eligible)': "cardiac_rehab_particpation",
                'HEALTHCARE (Cardiac Rehabilitation Units)': "cardiac_rehab_units",
                'HEALTHCARE (Cholesterol Screening)': "cholesterol_screen",
                'HEALTHCARE (Cholesterol-Lowering Medication - Nonadherence - All race': "cholesterol_medication",
                'HEALTHCARE (CVD Physicians)': "no_of_cvd_physicians",
                'HEALTHCARE (Emergency Services Centers)': "no_of_emercency_centers", 
                'HEALTHCARE (Hospitals #)': "no_of_hospitals",
                'HEALTHCARE (Insurance Status)': "insurance_status", 
                'HEALTHCARE (Neurological Services)': "neuro_services_avail",
                'HEALTHCARE (Neurologists)': "no_of_neurologists",
                'HEALTHCARE (Neurosurgeons)': "no_of_neurosurgeons",
                'HEALTHCARE (Pharmacies and Drug Stores)': "no_of_pharmacies",
                'HEALTHCARE (Primary Care Physicians)': "no_of_primary_physicians",                 
                'RISK (Diagnosed Diabetes)': "diagnosed_diabetes",
                'RISK (High Cholesterol Awareness)': "cholesterol_awareness",
                'RISK (Leisure Time Physical Inactivity)': "leisure_inactivity", 
                'RISK (Obesity)': "obesity",
                'RISK (SMOKING)': "smoking", 
                'SOCIAL (Air Quality PM2.5 Level)': "air_quality",
                'SOCIAL (Demo. Aged 65 and Older)': "population_age_65+",
                'SOCIAL (Demo. American Indian or Alaska Native)': "population_indians",
                'SOCIAL (Demo. Asian or Native Hawaiian or Other Pacific Island)': "population_asian",
                'SOCIAL (Demo. Black)': "population_black",
                'SOCIAL (Demo. Hispanic or Latino)': "population_hispanic",
                'SOCIAL (Demo. TotaL Population)': "population_total", 
                'SOCIAL (Demo. White)': "population_white",
                'SOCIAL (Income Inequality)': "income_inequality", 
                'SOCIAL (Median Household Income)': "median_houshld_income",
                'SOCIAL (Park Access)': "park_access", 
                'SOCIAL (Poverty %)': "poverty_rate",         
                'SOCIAL (Unemployment Rate)': "unemployment_rate", 
                'SOCIAL (Urban - Rural Status)': "urban_rural_class",
                'Heart Disease Prevalence': "prevalence",                
               }
len(replace_dict)

37

In [151]:
df1.rename(columns = replace_dict, inplace=True)
df1.index

Int64Index([ 2013,  2016,  2020,  2050,  2060,  2068,  2070,  2090,  2100,
             2105,
            ...
            72141, 72143, 72145, 72147, 72149, 72151, 72153, 78010, 78020,
            78030],
           dtype='int64', name='cnty_fips', length=3226)

In [152]:
df1.dtypes

county                           object
state                            object
cardiac_intensive_care_units      int64
cardiac_rehab_completion        float64
cardiac_rehab_eligibility       float64
cardiac_rehab_sessions          float64
cardiac_rehab_particpation      float64
cardiac_rehab_units               int64
cholesterol_screen              float64
cholesterol_medication          float64
no_of_cvd_physicians            float64
no_of_emercency_centers         float64
no_of_hospitals                   int64
insurance_status                float64
neuro_services_avail              int64
no_of_neurologists              float64
no_of_neurosurgeons             float64
no_of_pharmacies                float64
no_of_primary_physicians        float64
prevalence                      float64
diagnosed_diabetes              float64
cholesterol_awareness           float64
leisure_inactivity              float64
obesity                         float64
smoking                         float64


### Removing Missing Data

In [153]:
df2 = df1.dropna()
df2.shape

(3142, 39)

As shown above, if we simply dropped all the  NaNs in the dataset, we would have lost over 50% of the dataset. Insead, we systematically manipulate the data as follows.

### Estimating number of missing rows

In [154]:
df1 = df1.replace(-1, np.nan)
missing_df = df1.isna().sum().to_frame()
missing_df.columns = ["count"]
missing_df["percent"] = round((missing_df["count"]/3226)*100, 2) 
missing_df = missing_df.sort_values("percent", ascending = True)
missing_df

,count,percent
county,0,0.00
state,0,0.00
cardiac_intensive_care_units,2,0.06
neuro_services_avail,2,0.06
cardiac_rehab_units,2,0.06
no_of_hospitals,2,0.06
no_of_pharmacies,3,0.09
no_of_emercency_centers,3,0.09
population_age_65+,6,0.19
population_indians,6,0.19


In [155]:
# How about creating a function that repeats the above step whenever necessary..

def estimate_missing_rows(df):
    num_rows, no_cols = df.shape
    df = df.replace(-1, np.nan)
    missing_df = df.isna().sum().to_frame()
    missing_df.columns = ["count"]
    missing_df["percent"] = round((missing_df["count"]/num_rows)*100, 2) 
    missing_df = missing_df.sort_values("percent", ascending = True)
    
    return missing_df

#### Now we investigate the missing data in the remaining variables one by one.

###### Cardiac_intensive_care_units

In [156]:
dfx = df1[["county", "state", "cardiac_intensive_care_units"]]
display(dfx[dfx["cardiac_intensive_care_units"].isna() == True])

#Considering that American Samoa and Mariana Islands are unincorporated terrotory of the unitedstates, we will remove these datasets.
# We also check to confirm that this counties/states have no values for all the columns.

display(df1[df1["county"] == "American Samoa"])
display(df1[df1["county"] == "Mariana Islands"])

# Dropping both counties..
df1 = df1.drop([60000, 69000])

# check to see that these counties have been dropped..
df1["cardiac_intensive_care_units"].isna().sum()

# Output is 0. Good!
display(df1[df1["cardiac_intensive_care_units"].isna() == True])

,county,state,cardiac_intensive_care_units
cnty_fips,,,
60000,American Samoa,AS,NaN
69000,Mariana Islands,MP,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
60000,American Samoa,AS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69000,Mariana Islands,MP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,NaN,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


#### Perhaps this may be the case with some other variables.

##### neuro_services_avail

In [157]:
dfx = df1[["county", "state", "neuro_services_avail"]]
display(dfx[dfx["neuro_services_avail"].isna() == True])


,county,state,neuro_services_avail
cnty_fips,,,


#### "neuro_services_avail" no longer has any missing data. 

#### Perhaps this is the smae for other variables. So rather than checking for the variables one by one, we call the function estimate_missing_rows to check the percent of missing data again. In the following dataframe, we can see that the frequency of missing data has reduced with some now zeros. 


In [158]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
state,0,0.00
cardiac_intensive_care_units,0,0.00
neuro_services_avail,0,0.00
cardiac_rehab_units,0,0.00
no_of_hospitals,0,0.00
no_of_pharmacies,3,0.09
no_of_emercency_centers,3,0.09
population_age_65+,4,0.12
population_indians,4,0.12


#### Let's investigate another variable

##### no_of_pharmacies

In [159]:
dfx = df1[["county", "state", "no_of_pharmacies"]]
display(dfx[dfx["no_of_pharmacies"].isna() == True])

# We also check to confirm that this counties/states have no values for all the columns.

display(df1[df1["county"] == "Saint Croix (County Equivalent)"])
display(df1[df1["county"] == "Saint John (County Equivalent)"])
display(df1[df1["county"] == "Saint Thomas (County Equivalent)"])

# Since each of these counties have missing data for most of the columns, we drop these counties.
# Dropping both counties..
df1 = df1.drop([78010, 78020, 78030])

# check to see that these counties have been dropped..
df1["no_of_pharmacies"].isna().sum()

# Output is 0. Good!
display(df1[df1["no_of_pharmacies"].isna() == True])

,county,state,no_of_pharmacies
cnty_fips,,,
78010,Saint Croix (County Equivalent),VI,NaN
78020,Saint John (County Equivalent),VI,NaN
78030,Saint Thomas (County Equivalent),VI,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
78010,Saint Croix (County Equivalent),VI,0.0,NaN,5.7,21.3,18.6,0.0,NaN,44.2,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
78020,Saint John (County Equivalent),VI,0.0,NaN,NaN,NaN,NaN,0.0,NaN,43.8,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
78030,Saint Thomas (County Equivalent),VI,0.0,NaN,4.2,26.4,16.7,0.0,NaN,44.4,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


#### Let's check the frequencies again.

In [160]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
state,0,0.00
cardiac_intensive_care_units,0,0.00
no_of_pharmacies,0,0.00
neuro_services_avail,0,0.00
cardiac_rehab_units,0,0.00
no_of_hospitals,0,0.00
no_of_emercency_centers,0,0.00
population_age_65+,1,0.03
population_indians,1,0.03


#### Investigation continues..

###### population_age_65+

In [161]:
dfx = df1[["county", "state", "population_age_65+"]]
display(dfx[dfx["population_age_65+"].isna() == True])

# We also check to confirm that this counties/states have no values for all the columns.

display(df1[df1["county"] == "Guam"])


# Since each of these counties have missing data for most of the columns, we drop these counties.
# Dropping both counties..
df1 = df1.drop([66010])

# check to see that these counties have been dropped..
df1["population_age_65+"].isna().sum()

# Output is 0. Good!

,county,state,population_age_65+
cnty_fips,,,
66010,Guam,GU,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66010,Guam,GU,0.0,NaN,11.1,NaN,NaN,0.0,NaN,40.9,167.8,8.3,3.0,NaN,0.0,83.9,NaN,8.3,2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0

#### Let's check the frequencies again.

In [162]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
no_of_pharmacies,0,0.00
population_age_65+,0,0.00
neuro_services_avail,0,0.00
population_indians,0,0.00
no_of_hospitals,0,0.00
no_of_emercency_centers,0,0.00
diagnosed_diabetes,0,0.00
population_black,0,0.00
population_asian,0,0.00


#### Investigation continues..

###### smoking

In [163]:
dfx = df1[["county", "state", "smoking"]]
display(dfx[dfx["smoking"].isna() == True])

# We also check to confirm that this counties/states have no values for all the columns.

pd.set_option('display.max_columns', 50)
display(df1[df1["smoking"].isna() == True])

# There are 78 rows without data for smoking. 
# By looking at this variable, it is observed that, for all these 80 rows, the risk and health care factors are ..
# not listed. Infact, only some social factors were listed. 
# Interpolating over this amount of data over each variables may introduce bias into the dataset. So, we simply delete these counties.



# Dropping both counties..

df1 = df1[df1["smoking"].notna()]  



# check to see that these counties have been dropped..
df1["population_age_65+"].isna().sum()


,county,state,smoking
cnty_fips,,,
72001,Adjuntas,PR,NaN
72003,Aguada,PR,NaN
72005,Aguadilla,PR,NaN
72007,Aguas Buenas,PR,NaN
72009,Aibonito,PR,NaN
...,...,...,...
72145,Vega Baja,PR,NaN
72147,Vieques,PR,NaN
72149,Villalba,PR,NaN


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
72001,Adjuntas,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,29.4,17.3,28.9,0.0,NaN,0.0,NaN,NaN,28.9,2.2,NaN,15.0,NaN,NaN,NaN,NaN,NaN,18.6,0.0,0.0,0.0,99.6,17891.0,0.2,0.503,0.503,NaN,NaN,15.3,NaN
72003,Aguada,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,28.3,36.6,46.4,0.0,NaN,0.0,NaN,36.6,46.4,1.1,NaN,12.9,NaN,NaN,NaN,NaN,NaN,18.2,0.1,0.0,0.0,92.4,37954.0,6.7,0.501,0.501,NaN,NaN,10.1,NaN
72005,Aguadilla,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,32.3,50.6,45.5,1.0,NaN,0.0,50.6,NaN,45.5,0.9,NaN,14.0,NaN,NaN,NaN,NaN,NaN,20.9,0.0,0.1,0.1,97.5,52803.0,1.5,0.560,0.560,NaN,NaN,10.3,NaN
72007,Aguas Buenas,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,32.8,NaN,24.1,0.0,NaN,0.0,NaN,NaN,24.1,2.8,NaN,14.5,NaN,NaN,NaN,NaN,NaN,18.9,0.0,0.0,0.0,99.6,25748.0,0.3,0.518,0.518,NaN,NaN,9.2,NaN
72009,Aibonito,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,18.0,NaN,31.6,1.0,NaN,0.0,NaN,NaN,31.6,0.8,NaN,13.7,NaN,NaN,NaN,NaN,NaN,19.9,0.0,0.0,0.0,99.6,22988.0,0.1,0.477,0.477,NaN,NaN,10.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,Vega Baja,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,29.4,NaN,27.9,1.0,NaN,0.0,25.1,NaN,27.9,1.2,NaN,14.5,NaN,NaN,NaN,NaN,NaN,19.5,0.0,0.0,0.0,96.7,52192.0,2.9,0.501,0.501,NaN,NaN,9.6,NaN
72147,Vieques,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,34.8,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,8.4,NaN,13.6,NaN,NaN,NaN,NaN,NaN,21.4,0.0,0.0,0.0,94.9,8642.0,5.0,0.454,0.454,NaN,NaN,7.0,NaN
72149,Villalba,PR,0.0,NaN,NaN,NaN,NaN,0.0,NaN,22.5,21.5,27.9,0.0,NaN,0.0,21.5,NaN,27.9,5.4,NaN,14.6,NaN,NaN,NaN,NaN,NaN,16.7,0.0,0.0,0.0,99.7,22403.0,0.1,0.517,0.517,NaN,NaN,15.8,NaN


0

#### Let's check the frequencies again.

In [164]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
median_houshld_income,0,0.00
income_inequality,0,0.00
population_white,0,0.00
population_total,0,0.00
population_hispanic,0,0.00
population_black,0,0.00
population_asian,0,0.00
population_indians,0,0.00
population_age_65+,0,0.00


#### Considering the percentages of missing data in some of the remaining variables are relatively small, we can just drop the counties.


In [165]:
dfx = df1[["county", "state", "smoking"]]
display(dfx[dfx["smoking"].isna() == True])

# We also check to confirm that this counties/states have no values for all the columns.

pd.set_option('display.max_columns', 50)
display(df1[df1["smoking"].isna() == True])

# There are 78 rows without data for smoking. 
# By looking at this variable, it is observed that, for all these 80 rows, the risk and health care factors are ..
# not listed. Infact, only some social factors were listed. 
# Interpolating this amount over each variables may introduce bias into the dataset. So, we simply delete these counties.


# Dropping both counties..

df1 = df1.dropna(subset = ["poverty_rate","unemployment_rate","insurance_status","park_access","urban_rural_class"])  


# check the new size of the data
df1.shape

,county,state,smoking
cnty_fips,,,


,county,state,cardiac_intensive_care_units,cardiac_rehab_completion,cardiac_rehab_eligibility,cardiac_rehab_sessions,cardiac_rehab_particpation,cardiac_rehab_units,cholesterol_screen,cholesterol_medication,no_of_cvd_physicians,no_of_emercency_centers,no_of_hospitals,insurance_status,neuro_services_avail,no_of_neurologists,no_of_neurosurgeons,no_of_pharmacies,no_of_primary_physicians,prevalence,diagnosed_diabetes,cholesterol_awareness,leisure_inactivity,obesity,smoking,air_quality,population_age_65+,population_indians,population_asian,population_black,population_hispanic,population_total,population_white,income_inequality,median_houshld_income,park_access,poverty_rate,unemployment_rate,urban_rural_class
cnty_fips,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


(3139, 39)

#### Let's check the frequencies again.

In [166]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
cholesterol_awareness,0,0.00
leisure_inactivity,0,0.00
obesity,0,0.00
smoking,0,0.00
population_age_65+,0,0.00
population_indians,0,0.00
population_asian,0,0.00
population_black,0,0.00
population_hispanic,0,0.00


##### air_quality

In [167]:
dfx = df1[["county", "state", "air_quality"]]
dfx = dfx[dfx["air_quality"].isna() == True]
display(dfx["state"].unique())

array(['AK'], dtype=object)

Since the 24 counties with missing air_quality data are in Alaska, we obtain the average of the remaining counties in Alaska and populate it into the dataframe for counites in Alaska with missing data

In [168]:
dfx = df1[df1["state"] == "AK"]
avg = dfx["air_quality"].mean()
df1["air_quality"].fillna(avg, inplace = True)

# Check..
df1["air_quality"].isna().sum()

0

##### cholesterol_medication

Here just like we did for the air_quality, we fill NaN with the average value for each state.

In [169]:
dfx = df1[["county", "state", "cholesterol_medication"]]
dfx = dfx[dfx["cholesterol_medication"].isna() == True]
display(dfx["state"].unique())

array(['AK', 'CA', 'CO', 'ID', 'NV', 'UT', 'TX', 'KS', 'NE', 'NM', 'SD',
       'MT', 'ND', 'MS'], dtype=object)

In [170]:
missing_state = list(dfx["state"].unique())
df_ = df1[df1["state"].isin(missing_state)]
# dfx["state"].unique()               # Check..
for this_state in missing_state:
    dfx = df1[df1["state"] == this_state]
    avg = dfx["cholesterol_medication"].mean()
    df1["cholesterol_medication"].fillna(avg, inplace = True)
    
# check ..
df_["cholesterol_medication"].isna().sum()

83

In [171]:
missing_state

['AK',
 'CA',
 'CO',
 'ID',
 'NV',
 'UT',
 'TX',
 'KS',
 'NE',
 'NM',
 'SD',
 'MT',
 'ND',
 'MS']

In [172]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
cholesterol_awareness,0,0.00
leisure_inactivity,0,0.00
obesity,0,0.00
smoking,0,0.00
air_quality,0,0.00
population_age_65+,0,0.00
population_indians,0,0.00
diagnosed_diabetes,0,0.00
population_asian,0,0.00


In [173]:
df1.shape

(3139, 39)

##### We drop "cardiac_rehab_completion", "no_of_cvd_physicians", "no_of_neurologists", and "no_of_neurosurgeons" because they have more than 50% missing data.


In [174]:
drop_list = ["cardiac_rehab_completion", "no_of_cvd_physicians", "no_of_neurologists", "no_of_neurosurgeons"]

df1.drop(columns = drop_list, inplace = True)
df1.head()
df1.shape

(3139, 35)

In [175]:
estimate_missing_rows(df1)

,count,percent
county,0,0.00
obesity,0,0.00
smoking,0,0.00
air_quality,0,0.00
population_age_65+,0,0.00
population_indians,0,0.00
population_asian,0,0.00
leisure_inactivity,0,0.00
population_black,0,0.00
population_total,0,0.00


##### We want to keep the remaining variables as is. But the quantities of missing data are significantly large that we don't want to perform any data imputation so as not to introduce bias in the datasets. Therefore, we just drop remaining missing data in the dataset. 

In [176]:
df1.dropna(inplace = True)
df1.shape

(2310, 35)

#### Check data format and change as appropriate

In [177]:
df1.dtypes

county                           object
state                            object
cardiac_intensive_care_units    float64
cardiac_rehab_eligibility       float64
cardiac_rehab_sessions          float64
cardiac_rehab_particpation      float64
cardiac_rehab_units             float64
cholesterol_screen              float64
cholesterol_medication          float64
no_of_emercency_centers         float64
no_of_hospitals                 float64
insurance_status                float64
neuro_services_avail            float64
no_of_pharmacies                float64
no_of_primary_physicians        float64
prevalence                      float64
diagnosed_diabetes              float64
cholesterol_awareness           float64
leisure_inactivity              float64
obesity                         float64
smoking                         float64
air_quality                     float64
population_age_65+              float64
population_indians              float64
population_asian                float64


#### Convert "urban-rural-class" to categorical and get dummy variables

In [180]:
df1["urban_rural_class"] = df1["urban_rural_class"].astype('category')
df1["urban_rural_class"].unique()

[3.0, 4.0, 1.0, 2.0]
Categories (4, float64): [3.0, 4.0, 1.0, 2.0]

In [183]:
dummy_variable_1 = pd.get_dummies(df1["urban_rural_class"])

dummy_variable_1.rename(columns={1.0:'large_central_metro', 2.0:'large_fringe_metro', 3.0:'small_metro', 4.0:'nonmetro'}, inplace=True)
dummy_variable_1.head()

,large_central_metro,large_fringe_metro,small_metro,nonmetro
cnty_fips,,,,
2020,0,0,1,0
2090,0,0,1,0
2122,0,0,0,1
2170,0,0,1,0
6001,1,0,0,0


In [184]:
# merge data frame "df" and "dummy_variable_1" 
df1 = pd.concat([df1, dummy_variable_1], axis=1)

# drop original column "fuel-type" from "df"
df1.drop("urban_rural_class", axis = 1, inplace=True)

#### Export the cleaned data.

In [185]:
df1.to_csv("cdc_clean.csv", encoding="utf-8")